# Feature Engineering pour le Marketing Mix Modeling

Ce notebook présente le processus de création des caractéristiques pour le modèle MMM, incluant les transformations d'adstock, la saturation et les caractéristiques temporelles.

In [ ]:
# Importer les bibliothèques nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
import os
import sys

# Configuration de Matplotlib
plt.style.use('seaborn-whitegrid')
plt.rcParams['figure.figsize'] = (12, 7)
plt.rcParams['font.size'] = 12

# Ajouter le répertoire parent au chemin Python
sys.path.append('..')

In [ ]:
# Initialiser une session Spark
spark = SparkSession.builder \
    .appName("mmm_feature_engineering") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

print(f"Spark version: {spark.version}")

## 1. Chargement des données prétraitées

In [ ]:
# Charger les données des ventes quotidiennes
from src.data.online_retail_loader import OnlineRetailLoader
import json

# Charger la configuration
with open("../config/online_retail_config.json", "r") as f:
    config = json.load(f)

# Initialiser le loader
retail_loader = OnlineRetailLoader(spark, "../config/online_retail_config.json")

# Charger les données retail
print("Chargement des données retail...")
retail_df = retail_loader.load_retail_data()

# Créer les données de ventes quotidiennes
print("Création des données de ventes quotidiennes...")
daily_sales = retail_loader.create_daily_sales_data(retail_df)

# Afficher un aperçu
print("\nAperçu des données de ventes quotidiennes:")
daily_sales.show(5)

# Créer les données marketing simulées
print("\nCréation des données marketing...")
marketing_df = retail_loader.create_marketing_channel_data(retail_df)
print("\nAperçu des données marketing:")
marketing_df.show(5)

# Créer les facteurs externes
print("\nCréation des facteurs externes...")
external_df = retail_loader.create_external_factors(retail_df)
print("\nAperçu des facteurs externes:")
external_df.show(5)

## 2. Feature Engineering pour les ventes

In [ ]:
from src.features.feature_engineer import FeatureEngineer

# Initialiser l'ingénieur de caractéristiques
feature_engineer = FeatureEngineer("../config/online_retail_config.json")

# Créer des caractéristiques temporelles
print("Création des caractéristiques temporelles...")
sales_features = feature_engineer.create_seasonality_features(daily_sales)
sales_features = feature_engineer.create_holiday_features(sales_features)

# Afficher un aperçu des caractéristiques
print("\nAperçu des caractéristiques temporelles:")
sales_features.select("date", "revenue", "day_of_week", "month", "is_weekend", "month_sin", "month_cos").show(5)

## 3. Feature Engineering pour les canaux marketing

In [ ]:
# Créer des caractéristiques pour les canaux marketing
print("Création des caractéristiques pour les canaux marketing...")

# Créer des lag features
marketing_features = feature_engineer.create_lag_features(
    marketing_df, 
    id_cols=["channel"], 
    target_cols=["spend"],
    lag_periods=[1, 3, 7, 14, 28]
)

# Créer des rolling features
marketing_features = feature_engineer.create_rolling_features(
    marketing_features,
    id_cols=["channel"],
    target_cols=["spend"],
    windows=[7, 14, 30]
)

# Afficher un aperçu
print("\nAperçu des caractéristiques marketing:")
marketing_features.select("date", "channel", "spend", "spend_lag_7", "spend_avg_14d").show(5)

## 4. Modélisation de l'effet Adstock

In [ ]:
from src.models.adstock import AdstockModels
import pandas as pd

# Paramètres d'adstock par canal
adstock_params = {
    'tv': {'decay_rate': 0.7, 'max_lag': 14, 'saturation_type': 'hill', 'k': 0.7, 'S': 50000},
    'radio': {'decay_rate': 0.6, 'max_lag': 7, 'saturation_type': 'hill', 'k': 0.6, 'S': 20000},
    'print': {'decay_rate': 0.5, 'max_lag': 21, 'saturation_type': 'hill', 'k': 0.5, 'S': 30000},
    'social_media': {'decay_rate': 0.5, 'max_lag': 5, 'saturation_type': 'hill', 'k': 0.8, 'S': 25000},
    'search': {'decay_rate': 0.4, 'max_lag': 3, 'saturation_type': 'hill', 'k': 0.9, 'S': 40000},
    'email': {'decay_rate': 0.3, 'max_lag': 4, 'saturation_type': 'hill', 'k': 0.7, 'S': 10000},
    'display': {'decay_rate': 0.5, 'max_lag': 10, 'saturation_type': 'hill', 'k': 0.6, 'S': 20000}
}

# Pivoter les données marketing pour avoir une colonne par canal
print("Pivoter les données marketing...")
pivot_marketing = marketing_df.groupBy("date").pivot("channel").sum("spend").na.fill(0)

# Démonstration de l'effet adstock pour un canal
channel = 'tv'
if channel in pivot_marketing.columns:
    print(f"\nDémonstration de l'effet adstock pour le canal {channel}:")
    
    # Convertir en pandas pour la démonstration
    pdf = pivot_marketing.select("date", channel).toPandas()
    pdf["date"] = pd.to_datetime(pdf["date"])
    pdf = pdf.sort_values("date")
    pdf.set_index("date", inplace=True)
    
    # Appliquer l'adstock géométrique
    params = adstock_params[channel]
    adstock = AdstockModels.geometric_adstock(
        pdf, 
        channel,
        decay_rate=params["decay_rate"],
        max_lag=params["max_lag"]
    )
    
    # Appliquer la saturation
    transformed = AdstockModels.apply_saturation(
        adstock,
        saturation_type=params["saturation_type"],
        k=params["k"],
        S=params["S"]
    )
    
    # Créer un DataFrame pour visualiser
    result_pdf = pd.DataFrame({
        "date": pdf.index,
        "original": pdf[channel],
        "adstock": adstock,
        "transformed": transformed
    })
    
    # Visualiser l'effet
    plt.figure(figsize=(14, 8))
    plt.plot(result_pdf["date"], result_pdf["original"], label="Dépense originale", color="blue")
    plt.plot(result_pdf["date"], result_pdf["adstock"], label="Effet adstock", color="red")
    plt.plot(result_pdf["date"], result_pdf["transformed"], label="Effet avec saturation", color="green")
    plt.title(f"Transformation d'adstock et saturation pour {channel}")
    plt.xlabel("Date")
    plt.ylabel("Valeur")
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # Afficher la visualisation pour un mois seulement
    single_month = result_pdf.iloc[30:60]
    
    plt.figure(figsize=(14, 6))
    plt.plot(single_month["date"], single_month["original"], label="Dépense originale", color="blue")
    plt.plot(single_month["date"], single_month["adstock"], label="Effet adstock", color="red")
    plt.plot(single_month["date"], single_month["transformed"], label="Effet avec saturation", color="green")
    plt.title(f"Transformation d'adstock et saturation pour {channel} - Un mois")
    plt.xlabel("Date")
    plt.ylabel("Valeur")
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

## 5. Préparation des données finales pour la modélisation

In [ ]:
from src.models.mmm_model import MMMModel

# Initialiser le modèle MMM
mmm_model = MMMModel(spark, "../config/online_retail_config.json")

# Prétraiter les données pour la modélisation
print("Prétraitement des données pour la modélisation...")
preprocessed_df = mmm_model.preprocess_data(daily_sales, marketing_df, external_df)

# Afficher un aperçu
print("\nAperçu des données prétraitées:")
preprocessed_df.select([col for col in preprocessed_df.columns if col in ["date", "revenue", "tv", "radio", "tv_adstock", "is_holiday"]]).show(5)

# Sauvegarder les données prétraitées
preprocessed_df.write.mode("overwrite").parquet("../data/mmm_features.parquet")
print("\nDonnées prétraitées sauvegardées avec succès!")

# Nombre de lignes et de colonnes
rows = preprocessed_df.count()
cols = len(preprocessed_df.columns)
print(f"Dimensions finales: {rows} lignes, {cols} colonnes")

## 6. Analyse des corrélations

In [ ]:
# Convertir en pandas pour l'analyse des corrélations
print("Analyse des corrélations...")
pdf = preprocessed_df.toPandas()

# Sélectionner les colonnes pertinentes
cols_to_analyze = ["revenue"] + \
                  [col for col in pdf.columns if col in config["marketing_channels"]] + \
                  [col for col in pdf.columns if "_adstock" in col] + \
                  ["consumer_confidence", "gdp_growth", "unemployment", "is_holiday", "is_promo"]

# Créer une matrice de corrélation
corr_matrix = pdf[cols_to_analyze].corr()

# Visualiser la matrice de corrélation
plt.figure(figsize=(16, 12))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", vmin=-1, vmax=1, center=0, fmt=".2f")
plt.title("Matrice de corrélation des caractéristiques principales")
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Corrélation avec la revenue
revenue_corr = corr_matrix["revenue"].sort_values(ascending=False)
print("\nCorrélation des caractéristiques avec la revenue:")
print(revenue_corr)

## 7. Analyse de l'importance des caractéristiques

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

# Préparer les données pour l'analyse d'importance
print("Analyse de l'importance des caractéristiques...")
X = pdf.drop(["date", "revenue"], axis=1)
y = pdf["revenue"]

# Diviser en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Créer et entraîner un modèle LightGBM
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1
}

# Créer le dataset LightGBM
train_data = lgb.Dataset(X_train, label=y_train)
model = lgb.train(params, train_data, num_boost_round=100)

# Extraire l'importance des caractéristiques
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': model.feature_importance(importance_type='gain')
}).sort_values('importance', ascending=False)

# Visualiser l'importance des caractéristiques
plt.figure(figsize=(14, 10))
sns.barplot(x='importance', y='feature', data=feature_importance.head(20))
plt.title('Top 20 des caractéristiques les plus importantes')
plt.xlabel('Importance (gain)')
plt.ylabel('Caractéristique')
plt.tight_layout()
plt.show()

print("\nTop 20 des caractéristiques les plus importantes:")
print(feature_importance.head(20))

## 8. Préparation pour la modélisation

In [ ]:
# Diviser les données en ensembles d'entraînement et de test
train_end_date = config['data']['train_end_date']
test_start_date = config['data']['test_start_date']

print(f"Division en ensembles d'entraînement (jusqu'au {train_end_date}) et de test (à partir du {test_start_date})...")
train_df = preprocessed_df.filter(f"date <= '{train_end_date}'")
test_df = preprocessed_df.filter(f"date >= '{test_start_date}'")

# Sauvegarder les ensembles d'entraînement et de test
train_df.write.mode("overwrite").parquet("../data/train_data.parquet")
test_df.write.mode("overwrite").parquet("../data/test_data.parquet")

print(f"Ensemble d'entraînement: {train_df.count()} lignes")
print(f"Ensemble de test: {test_df.count()} lignes")

print("\nPréparation des données terminée! Prêt pour la modélisation.")

In [ ]:
# Arrêter la session Spark
spark.stop()